In [4]:
import cvxpy as cp
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('player_prices_w_pred_Ben.csv')
data.head(50)

,Unnamed: 0,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,lgbm_projection,Linear_DFS_Prediction,NN_DFS_Prediction
0,0,G,Jordan Binnington (13678944),Jordan Binnington,13678944,G,8400,CLS@STL 11/01/2019 08:00PM ET,STL,15.68,14.82,13.629441,12.577880
1,1,G,Philipp Grubauer (13678945),Philipp Grubauer,13678945,G,8200,DAL@COL 11/01/2019 09:00PM ET,COL,16.70,13.33,0.000000,0.000000
2,2,LW,Alex Ovechkin (13678140),Alex Ovechkin,13678140,W/UTIL,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,19.41,12.89,15.984278,15.968162
3,3,G,Braden Holtby (13678946),Braden Holtby,13678946,G,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,10.99,13.90,13.269326,11.892563
4,4,G,Petr Mrazek (13678947),Petr Mrazek,13678947,G,8000,DET@CAR 11/01/2019 07:30PM ET,CAR,14.68,12.60,0.000000,0.000000
5,5,C,Nathan MacKinnon (13677890),Nathan MacKinnon,13677890,C/UTIL,8000,DAL@COL 11/01/2019 09:00PM ET,COL,16.45,15.82,14.774305,14.399432
6,6,G,Andrei Vasilevskiy (13678949),Andrei Vasilevskiy,13678949,G,7900,TB@NYI 11/01/2019 07:00PM ET,TB,15.72,13.78,14.412698,12.497589
7,7,G,Martin Jones (13678948),Martin Jones,13678948,G,7800,WPG@SJ 11/01/2019 10:30PM ET,SJ,9.00,14.06,0.000000,0.000000
8,8,G,John Gibson (13678950),John Gibson,13678950,G,7700,VAN@ANH 11/01/2019 10:00PM ET,ANH,15.19,13.88,0.000000,0.000000
9,9,G,Carter Hart (13678951),Carter Hart,13678951,G,7600,PHI@NJ 11/01/2019 07:00PM ET,PHI,4.97,9.11,0.000000,0.000000


In [6]:
#Test of forcing player selection
already_selected=[]
#already_selected = ['Ilya Samsonov','Elias Pettersson', 'Alex Ovechkin']

already_selected_data = data.loc[data['Name'].isin(already_selected)]

C_selected = already_selected_data.loc[already_selected_data['Position']=='C'].count()['Position']
D_selected = already_selected_data.loc[already_selected_data['Position']=='D'].count()['Position']
W_selected = already_selected_data.loc[already_selected_data['Position'].isin(['LW','RW'])].count()['Position']
G_selected = already_selected_data.loc[already_selected_data['Position']=='G'].count()['Position']
total_selected = C_selected + W_selected + D_selected + G_selected
selected_salary = np.sum(already_selected_data['Salary'])

data = data.loc[~data['Name'].isin(already_selected)]

Create Variable arrays for model

In [7]:
salary = np.array(data['Salary'])
projected_DFS = np.array(data['NN_DFS_Prediction'])
center = np.array(data['Position']=='C')
winger = np.array(np.logical_or(data['Position']=='LW',data['Position']=='RW'))
defense = np.array(data['Position']=='D')
goalie = np.array(data['Position']=='G')
selection = cp.Variable(len(salary), boolean=True)
budget = 50000-selected_salary
max_players = 8-total_selected

Create Constraints

In [8]:
budget_constraint = salary*selection <= budget
player_constraint = sum(selection) == max_players
center_min = selection*center >=2-C_selected
center_max = selection*center <=3-C_selected
winger_min = selection*winger >=2-W_selected
winger_max = selection*winger <=3-W_selected
defender_min = selection*defense >=2-D_selected
defender_max = selection*defense <=3-D_selected
goalie_constraint = selection*goalie == 1-G_selected

Objective Function

In [9]:
total_projected_value = projected_DFS * selection
objective = cp.Problem(cp.Maximize(total_projected_value), [budget_constraint,player_constraint, center_min, center_max, winger_min, winger_max, defender_min, defender_max, goalie_constraint])

In [10]:
objective.solve()

98.68757000144832

Create Optimal Player List

In [11]:
opt_selection = selection.value >= 0.9
player_list = data['Name'][opt_selection].append(already_selected_data['Name'])
opt_positions = data['Position'][opt_selection].append(already_selected_data['Position'])
opt_salary = data['Salary'][opt_selection].append(already_selected_data['Salary'])
print(player_list.tolist())
print(opt_positions)
print(np.sum(opt_salary))
print(type(player_list.tolist()))

['Alex Ovechkin', 'Nathan MacKinnon', 'John Carlson', 'Jimmy Howard', 'Dylan Larkin', 'Blake Wheeler', 'Gustav Nyquist', 'Samuel Girard']
2      LW
5       C
15      D
21      G
35      C
52     RW
148    LW
156     D
Name: Position, dtype: object
49800
<class 'list'>
